**감성분석**

- 문서의 주관적인 감성/의견/감정/기분등을 파악하기 위한 방법
- 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등의 분야에서 활용됨
- 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성(Sentiment) 수치를 계산함
- 긍정 감성 지수와 부정 감성 지수로 구성됨





In [1]:
# 1. 지도학습 기반 감성 분석

import pandas as pd

- id : 각 데이터의 id
- sentiment : 영화평(review)의 Sentiment 결과값 (1은 긍정 / 0은 부정)
- review : 영화평의 텍스트 내용

In [3]:
review_df = pd.read_csv('C:\workspaces\Sentiment Analysis\labeledTrainData.tsv', header=0, sep="\t", quoting=3)
review_df

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."


In [4]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [5]:
# 태그, 영어가 아닌 숫자나 특수문자는 모두 공란으로 변경해야 함
# 정규표현식 사용

import re

In [6]:
# <br> html 태그가 있다면 공백으로 replace 하기
review_df['review'] = review_df['review'].str.replace('<br/>', ' ')

In [7]:
# re를 이용해서 영어 문자열이 아니면 모두 공백으로 바꾸기
review_df['review'] = review_df['review'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))

In [9]:
# 결정 값 클래스인 sentiment 칼럼을 별도로 추출한 뒤 결정값 데이터 세트 만들기
from sklearn.model_selection import train_test_split

class_df = review_df["sentiment"]

In [10]:
# 원본 데이터 세트에서 id와 sentiment 칼럼 삭제 해서 피터 데이터 세트 만들기
feature_df = review_df.drop(['id', 'sentiment'], axis=1, inplace=False)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

# 학습용 데이터는 17,500개의 리뷰
# 테스트용 데이터는 7,500개의 리뷰
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [26]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# CountVectorizer : 문서 집합을 토큰으로 나누고, 토큰의 출현 빈도를 세어 Bag of Words를 만듦
# TfidfTransformer : TF-IDF(Term Frequency-Inverse Document Frequency)를 계산하여 단어에 가중치 부여 (단어의 상대적 중요성)


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score


### 1) Count 벡터화를 적용해서 예측 성능 측정

In [22]:
# Pipeline 클래스를 사용하여 텍스트 데이터를 처리하고 분류 모델을 적용하는 파이프라인을 만듦
# ngram_range=(1,2) : 최소 길이가 1이고 최대 길이가 2인 n-gram을 사용
# C : 로지스틱 회귀 모델의 규제 매개변수 (C=10은 중간 정도의 규제)
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lc_clf', LogisticRegression(solver='liblinear', C=10))
])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])

# 이진 분류 문제이므로  [:, 1]는 모든 행(:)에 대해 두 번째 열(인덱스 1)을 선택
# 모델이 예측한 각 샘플이 클래스 1에 속할 확률을 나타내는 값들의 배열을 생성
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print(f'예측 정확도 : {accuracy_score(y_test, pred)}')
print(f'ROC-AUC : {roc_auc_score(y_test, pred_probs)}')

예측 정확도 : 0.8869333333333334
ROC-AUC : 0.9505469639198725


### 2) TF-IDF 벡터화를 적용해 예측 성능 측정

In [27]:
# C : 로지스틱 회귀 모델의 규제 매개변수 (C=10은 중간 정도의 규제)
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lc_clf', LogisticRegression(solver='liblinear', C=10))
])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print(f'예측 정확도 : {accuracy_score(y_test, pred)}')
print(f'ROC-AUC : {roc_auc_score(y_test, pred_probs)}')

예측 정확도 : 0.8932
ROC-AUC : 0.9600004979512861
